## Lab 2: Memory を追加してエージェントをパーソナライズ

### 概要

Lab 1 では、単一ユーザーのローカルセッションで正常に動作するカスタマーサポートエージェントを構築しました。しかし、実際のカスタマーサポートでは、ローカル環境で実行される単一ユーザーを超えてスケールする必要があります。

**本番環境でエージェントを実行する**場合、以下が必要になります：
- **マルチユーザーサポート**: 数千の顧客を同時に処理
- **永続的ストレージ**: セッションのライフサイクルを超えて会話を保存
- **長期学習**: 顧客の好みや行動パターンを抽出
- **クロスセッションの継続性**: 異なるインタラクション間で顧客を記憶

**ワークショップの進捗:**
- **Lab 1（完了）**: エージェントプロトタイプの作成 - 機能するカスタマーサポートエージェントを構築
- **Lab 2（現在）**: Memory による強化 - 会話コンテキストとパーソナライゼーションを追加
- **Lab 3**: Gateway と Identity によるスケーリング - エージェント間でツールを安全に共有
- **Lab 4**: 本番環境へのデプロイ - AgentCore Runtime を使用してオブザーバビリティを実現
- **Lab 5**: エージェントパフォーマンスの評価 - オンライン評価で品質を監視
- **Lab 6**: ユーザーインターフェースの構築 - 顧客向けアプリケーションを作成

このLabでは、あなたの金魚エージェント（数秒で会話を忘れる）をスマートでパーソナライズされたアシスタントに変換する、欠けている永続性と学習レイヤーを追加します。

Memory は知性の重要な構成要素です。大規模言語モデル（LLM）には印象的な能力がありますが、会話間で永続的なMemoryがありません。[Amazon Bedrock AgentCore Memory](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/memory-getting-started.html) は、AI エージェントが時間を超えてコンテキストを維持し、重要な事実を記憶し、一貫したパーソナライズされた体験を提供できるようにするマネージドサービスを提供することで、この制限に対処します。

AgentCore Memory は2つのレベルで動作します：
- **短期Memory**: 単一のインタラクションまたは密接に関連するセッション内で継続性を提供する、即時の会話コンテキストとセッションベースの情報
- **長期Memory**: 複数の会話にわたって抽出・保存される永続的な情報で、時間をかけてパーソナライズされた体験を可能にする事実、好み、要約を含む

### Lab 2 のアーキテクチャ
<div style="text-align:left">
    <img src="images/architecture_lab2_memory.png" width="75%"/>
</div>

*永続的な短期および長期Memory機能を備えたマルチユーザーエージェント*

### 前提条件

* 適切な権限を持つ **AWS アカウント**
* ローカルにインストールされた **Python 3.10+**
* 認証情報が設定された **AWS CLI**
* [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html) で有効化された **Anthropic Claude 3.7**
* 次のセルでインストールする **Strands Agents** およびその他のライブラリ
* AWS ワークショップアカウント内で作成される以下のリソース
    - AWS Lambda 関数 
    - AWS Lambda 実行 IAM ロール
    - AgentCore Gateway IAM ロール
    - AWS Lambda 関数が使用する DynamoDB テーブル
    - Cognito User Pool と User Pool Client

### ステップ 1: ライブラリのインポート

AgentCore Memory 用のライブラリをインポートしましょう。これには、AgentCore 機能を扱うための軽量なラッパーである [Amazon Bedrock AgentCore Python SDK](https://github.com/aws/bedrock-agentcore-sdk-python) を使用します。

In [ ]:
import logging
from boto3.session import Session

from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

from lab_helpers.utils import put_ssm_parameter

boto_session = Session()
REGION = boto_session.region_name

logger = logging.getLogger(__name__)

### ステップ 2: Bedrock AgentCore Memory リソースの作成

Amazon Bedrock AgentCore Memory は、AI エージェントに永続的なMemory機能を提供するフルマネージドサービスです。

#### AgentCore Memory の概念:

1. **短期Memory（STM）**: セッション内で会話コンテキストを即座に保存
2. **長期Memory（LTM）**: STM を非同期で処理し、意味のあるパターン、好み、事実を抽出
3. **Memory ストラテジー**: 情報の抽出と整理のための異なるアプローチ：
   - **USER_PREFERENCE**: 顧客の好み、行動、パターンを学習
   - **SEMANTIC**: ベクトル埋め込みを使用して事実情報を保存し、類似性検索を実現
4. **名前空間**: 顧客とコンテキストタイプによるMemoryの論理的なグループ化。以下の2つの名前空間を作成します：
- `support/customer/{actorId}/preferences`: 顧客の好みと行動パターン
- `support/customer/{actorId}/semantic`: 事実情報と会話履歴

この構造により、各顧客の情報が分離され、簡単に取得できるマルチテナントMemoryが実現します。

#### Memory 作成プロセス:

Memory リソースの作成には、基盤となるインフラストラクチャ（ベクトルデータベース、処理パイプラインなど）のプロビジョニングが含まれます。AWS がバックグラウンドでマネージドサービスをセットアップするため、通常2〜3分かかります。

In [ ]:
memory_name = "CustomerSupportMemory"

memory_manager = MemoryManager(region_name=REGION)
memory = memory_manager.get_or_create_memory(
    name=memory_name,
    strategies=[
                {
                    StrategyType.USER_PREFERENCE.value: {
                        "name": "CustomerPreferences",
                        "description": "Captures customer preferences and behavior",
                        "namespaces": ["support/customer/{actorId}/preferences"],
                    }
                },
                {
                    StrategyType.SEMANTIC.value: {
                        "name": "CustomerSupportSemantic",
                        "description": "Stores facts from conversations",
                        "namespaces": ["support/customer/{actorId}/semantic"],
                    }
                },
            ]
)
memory_id = memory["id"]
put_ssm_parameter("/app/customersupport/agentcore/memory_id", memory_id)

In [ ]:
if memory_id:
    print("✅ AgentCore Memory が正常に作成されました！")
    print(f"メモリ ID: {memory_id}")
else:
    print("メモリリソースが作成されませんでした。再試行してください！")

## ステップ 3: 過去の顧客インタラクションをシード

**なぜMemoryをシードするのか？**

本番環境では、エージェントは顧客とのインタラクションを通じて自然にMemoryを蓄積します。しかし、このLabでは、実際の会話を待たずに長期Memory（LTM）がどのように機能するかを示すために、過去の会話をシードしています。

**Memory処理の仕組み:**
1. `create_event` はインタラクションを**短期Memory**（STM）に即座に保存
2. STM は**長期Memory**ストラテジーによって非同期で処理される
3. LTM は将来の取得のためにパターン、好み、事実を抽出

実際の動作を確認するために、いくつかの顧客履歴をシードしましょう：

In [ ]:
from lab_helpers.lab2_memory import ACTOR_ID


# Seed with previous customer interactions
previous_interactions = [
    ("I'm having issues with my MacBook Pro overheating during video editing.", "USER"),
    (
        "I can help with that thermal issue. For video editing workloads, let's check your Activity Monitor and adjust performance settings. Your MacBook Pro order #MB-78432 is still under warranty.",
        "ASSISTANT",
    ),
    (
        "What's the return policy on gaming headphones? I need low latency for competitive FPS games",
        "USER",
    ),
    (
        "For gaming headphones, you have 30 days to return. Since you're into competitive FPS, I'd recommend checking the audio latency specs - most gaming models have <40ms latency.",
        "ASSISTANT",
    ),
    (
        "I need a laptop under $1200 for programming. Prefer 16GB RAM minimum and good Linux compatibility. I like ThinkPad models.",
        "USER",
    ),
    (
        "Perfect! For development work, I'd suggest looking at our ThinkPad E series or Dell XPS models. Both have excellent Linux support and 16GB RAM options within your budget.",
        "ASSISTANT",
    ),
]

# Save previous interactions
if memory_id:
    try:
        memory_client = MemoryClient(region_name=REGION)
        memory_client.create_event(
            memory_id=memory_id,
            actor_id=ACTOR_ID,
            session_id="previous_session",
            messages=previous_interactions,
        )
        print("✅ 顧客履歴のシードが正常に完了しました")
        print("📝 インタラクションが短期メモリに保存されました")
        print("⏳ 長期メモリの処理が自動的に開始されます...")
    except Exception as e:
        print(f"⚠️ 履歴のシード中にエラー: {e}")

### Memory処理の理解

`create_event` でイベントを作成した後、AgentCore Memory は2段階でデータを処理します：

1. **即時**: メッセージが短期Memory（STM）に保存される
2. **非同期**: STM が長期Memory（LTM）ストラテジーに処理される

LTM の処理には通常20〜30秒かかります。システムが以下を行うためです：
- 会話パターンの分析
- 顧客の好みと行動の抽出
- 事実情報のセマンティック埋め込みの作成
- 効率的な取得のための名前空間によるMemoryの整理

長期Memory処理が完了したかどうかを、顧客の好みを取得して確認しましょう：

In [ ]:
import time

# Wait for Long-Term Memory processing to complete
print("🔍 処理済みの長期メモリを確認中...")
retries = 0
max_retries = 6  # 1 minute wait

while retries < max_retries:
    memories = memory_client.retrieve_memories(
        memory_id=memory_id,
        namespace=f"support/customer/{ACTOR_ID}/preferences",
        query="can you summarize the support issue",
    )

    if memories:
        print(
            f"✅ {retries * 10} 秒後に {len(memories)} 件の好みメモリが見つかりました！"
        )
        break

    retries += 1
    if retries < max_retries:
        print(
            f"⏳ 処理中... あと10秒待機します（試行 {retries}/{max_retries}）"
        )
        time.sleep(10)
    else:
        print(
            "⚠️ メモリ処理に予想以上の時間がかかっています。過負荷の可能性があります..."
        )
        break

print(
    "🎯 AgentCore Memory がシードされた会話から自動的に抽出した顧客の好み:"
)
print("=" * 80)

for i, memory in enumerate(memories, 1):
    if isinstance(memory, dict):
        content = memory.get("content", {})
        if isinstance(content, dict):
            text = content.get("text", "")
            print(f"  {i}. {text}")

### セマンティックMemoryの探索

セマンティックMemoryは、ベクトル埋め込みを使用して会話からの事実情報を保存します。これにより、関連する事実とコンテキストの類似性ベースの取得が可能になります。

In [ ]:
import time

# Retrieve semantic memories (factual information)
while True:
    semantic_memories = memory_client.retrieve_memories(
        memory_id=memory_id,
        namespace=f"support/customer/{ACTOR_ID}/semantic",
        query="information on the technical support issue",
    )
    print("🧠 AgentCore Memory が会話から特定した事実情報:")
    print("=" * 80)
    if semantic_memories:
        break
    time.sleep(10)
for i, memory in enumerate(semantic_memories, 1):
    if isinstance(memory, dict):
        content = memory.get("content", {})
        if isinstance(content, dict):
            text = content.get("text", "")
            print(f"  {i}. {text}")

## ステップ 4: Memory を備えたカスタマーサポートエージェントの作成

次に、Lab 1 と同様にカスタマーサポートエージェントを実装しますが、今回はエージェントのセッションマネージャーに AgentCoreMemoryConfig を追加します。これにより、エージェントは新しいイベントを作成し、Memoryを取得できるようになります。

In [ ]:
import uuid

from strands import Agent
from strands.models import BedrockModel
from bedrock_agentcore.memory.integrations.strands.config import AgentCoreMemoryConfig, RetrievalConfig
from bedrock_agentcore.memory.integrations.strands.session_manager import AgentCoreMemorySessionManager

from lab_helpers.lab1_strands_agent import (
    SYSTEM_PROMPT,
    get_return_policy,
    web_search,
    get_product_info,
    get_technical_support,
    MODEL_ID,
)

session_id = uuid.uuid4()

memory_config = AgentCoreMemoryConfig(
        memory_id=memory_id,
        session_id=str(session_id),
        actor_id=ACTOR_ID,
        retrieval_config={
            "support/customer/{actorId}/semantic": RetrievalConfig(top_k=3, relevance_score=0.2),
            "support/customer/{actorId}/preferences": RetrievalConfig(top_k=3, relevance_score=0.2)
        }
    )

# Initialize the Bedrock model (Anthropic Claude 3.7 Sonnet)
model = BedrockModel(model_id=MODEL_ID, region_name=REGION)

# Create the customer support agent with all 5 tools
agent = Agent(
    model=model,
    session_manager=AgentCoreMemorySessionManager(memory_config, REGION),
    tools=[
        get_product_info,  # Tool 1: Simple product information lookup
        get_return_policy,  # Tool 2: Simple return policy lookup
        web_search,
        get_technical_support,
    ],
    system_prompt=SYSTEM_PROMPT,
)

## ステップ 5: パーソナライズされたエージェントのテスト

Memory強化されたエージェントをテストしましょう！エージェントが顧客の過去の好みを使用してパーソナライズされた推奨事項を提供する様子を観察してください。

エージェントは自動的に以下を行います：
1. Memoryから関連する顧客コンテキストを取得
2. そのコンテキストを使用してレスポンスをパーソナライズ
3. この新しいインタラクションを将来の使用のために保存

In [ ]:
print("🎧 顧客メモリを使用してヘッドフォンの推奨をテスト中...\n\n")
response1 = agent("Which headphones would you recommend?")

In [ ]:
print("\n💻 ノートPC の好みの記憶をテスト中...\n\n")
response2 = agent("What is my preferred laptop brand and requirements?")

エージェントが以下を記憶していることに注目してください：
- ゲームに関する好み（低遅延ヘッドフォン）
- ノートPCに関する好み（ThinkPad、16GB RAM、Linux互換性）
- 予算の制約（ノートPCは1200ドル）
- 以前の技術的な問題（MacBookの過熱）

これが AgentCore Memory の力です - 永続的でパーソナライズされた顧客体験！

## おめでとうございます！

**Lab 2: カスタマーサポートエージェントへの Memory の追加**を正常に完了しました！

### 達成したこと:

- Amazon Bedrock AgentCore Memory を使用してサーバーレスのマネージドMemoryを作成
- ユーザー設定とセマンティック（事実）情報を保存する長期Memoryを実装
- Strands Agents が提供するセッション管理メカニズムを使用して AgentCore Memory をカスタマーサポートエージェントと統合

##### 次のステップ [Lab 3 - Gateway と Identity によるスケーリング →](lab-03-agentcore-gateway.ipynb)

## リソース
- [Amazon Bedrock Agent Core Memory](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/memory.html)
- [Amazon Bedrock AgentCore Memory 詳細ブログ](https://aws.amazon.com/blogs/machine-learning/amazon-bedrock-agentcore-memory-building-context-aware-agents/)
- [Strands Agent SDK - AgentCore Memory サンプル](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/strands-sdk-memory.html)